In [6]:
"""
Steps to replicate derived property in holding domain on aggregation api returns different value. 

1. Define portfolio property. (which the derived property depends on)
2. Create portfolio with property.
3. upsert instruments.
4. Upsert transaction.
5. Upsert quote.
6. Upsert recipe.
7. Create holding derv property.
8. Do valuation to check values from these properties.
9. Check against holding API.

"""


"""
------------------------
Creating an API Factory:
------------------------

"""

# Import LUSID SDK
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory

# Import supporting LUSID utility packages
from lusidjam import RefreshingToken
#from fbnsdkutilities import ApiClientFactory

# Import other non-LUSID packages
import json
import pandas as pd
from datetime import datetime, timedelta, timezone
import os


# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")

print(f'Secrets path is {secrets_path}')

# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="DerivedPropertiesNotOnHoldingProviderTest"
)

print('LUSID Environment Initialised')

# Define a portfolio and parameters.
portfolio_code = "DERV_PROP_TEST_PORTFOLIO"
portfolio_name = "Derived Property Test Portfolio"

scope = "DERV_PROP_TEST"
transaction_scope = "DERV_PROP_TEST"
domain = "Portfolio"

# Time and date parameters in normalised format.
date_format = '%Y-%m-%d'


"""
---------------
Create Property
---------------

"""

# Create underlying value property in the portfolio domain.

properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

portfolio_scope = "Test"
properties = ["Schema"]

for property_code in properties:
    try:
        property_creation = properties_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain="Portfolio",
                scope=scope,
                code=property_code,
                display_name=property_code,
                data_type_id=models.ResourceId(code="string", scope="system"),
            )
        )

        print(property_creation)

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

# Create derived property that uses this value property.
derived_property_code = 'HoldingFromPortfolioDerivedProperty'
derived_property_formula = f"if ( Properties[Portfolio/DERV_PROP_TEST/Schema] eq 'JC_UK_Demo2_Schema' )\nthen 'IsJC_UK_Demo2_Schema'\nelse 'NotJC_UK_Demo2_Schema'"

try:
    property_creation = properties_api.create_derived_property_definition(
        create_derived_property_definition_request=models.CreateDerivedPropertyDefinitionRequest(
            domain="Holding",
            scope=scope,
            code=derived_property_code,
            display_name=derived_property_code,
            data_type_id=models.ResourceId(code="string", scope="system"),
            derivation_formula = derived_property_formula
        )
    )

    print(property_creation)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Secrets path is C:/Users/JoeCockings/OneDrive - Finbourne Technology Limited/Repos/secrets.json
LUSID Environment Initialised
Error creating Property Definition 'Portfolio/DERV_PROP_TEST/Schema' because it already exists.
{'code': 'HoldingFromPortfolioDerivedProperty',
 'collection_type': None,
 'constraint_style': None,
 'data_type_id': {'code': 'string', 'scope': 'system'},
 'derivation_formula': 'if ( Properties[Portfolio/DERV_PROP_TEST/Schema] eq '
                       "'JC_UK_Demo2_Schema' )\n"
                       "then 'IsJC_UK_Demo2_Schema'\n"
                       "else 'NotJC_UK_Demo2_Schema'",
 'display_name': 'HoldingFromPortfolioDerivedProperty',
 'domain': 'Holding',
 'href': None,
 'key': 'Holding/DERV_PROP_TEST/HoldingFromPortfolioDerivedProperty',
 'life_time': 'Perpetual',
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/PropertyDefinition',
            'method': 'GET',
            'relation': 'EntitySchema'